In [1]:
!pip install pyspark


# TD 1 - Transformations et Actions sur RDDs avec PySpark

Dans ce TD, nous allons explorer l'utilisation de l'API RDD de **PySpark** pour effectuer des opérations de manipulation et d'analyse de données. Ce TD vous permettra de pratiquer différentes transformations et actions sur des données distribuées.

Nous introduirons également un exercice utilisant **Spark SQL** pour comparer l'utilisation des RDDs et des DataFrames dans des scénarios d'analyse de données.

## Objectifs :
- Comprendre et manipuler les RDDs avec PySpark.
- Appliquer des transformations et actions sur les données.
- Utiliser Spark SQL pour effectuer des requêtes analytiques.



## Initialisation de SparkContext

Le **SparkContext** est nécessaire pour initialiser et gérer toutes les opérations dans Spark. Assurez-vous que votre SparkContext est bien créé avant de manipuler des RDDs.


In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

## Exercice 1 : Transformations et Actions

Nous avons un jeu de données simple contenant des informations sur des livres, avec le format suivant :
(titre, auteur, année de publication, genre, prix).


In [3]:
books = [
    ("Les Misérables", "Victor Hugo", 1862, "Roman", 12.99),
    ("Le Rouge et le Noir", "Stendhal", 1830, "Roman", 9.99),
    ("Candide", "Voltaire", 1759, "Philosophie", 8.99),
    ("L'Étranger", "Albert Camus", 1942, "Roman", 10.99),
    ("Le Père Goriot", "Honoré de Balzac", 1834, "Roman", 7.99),
    ("Discours de la méthode", "René Descartes", 1637, "Philosophie", 15.99),
]

### 1 - Créer un RDD à partir du jeu de données books

In [4]:
books_rdd = sc.parallelize(books)

### 2 - Utilisez map pour extraire uniquement les titres des livres et collecter le résultat avec collect.
1. map est une transformation ou une action ?
2. Quelle est la nature de l’opération map (narrow ou wide) ?
3. Si le volume de données augmente quel est risque d'utiliser l'opération collect ?



	    

In [5]:
books_rdd.map(lambda x : x[0]).collect()
# 2 - Opération Narrow
# 3 - En cas d'augmentation du volume de données, il existe un risque de rencontrer une erreur de "Out of Memory",
# car le dataset pourrait ne plus tenir dans la mémoire RAM du driver.

['Les Misérables',
 'Le Rouge et le Noir',
 'Candide',
 "L'Étranger",
 'Le Père Goriot',
 'Discours de la méthode']

### 3 - Filtrez les livres publiés après 1800.

In [6]:
books_rdd.filter(lambda x:x[2] > 1800).collect()

[('Les Misérables', 'Victor Hugo', 1862, 'Roman', 12.99),
 ('Le Rouge et le Noir', 'Stendhal', 1830, 'Roman', 9.99),
 ("L'Étranger", 'Albert Camus', 1942, 'Roman', 10.99),
 ('Le Père Goriot', 'Honoré de Balzac', 1834, 'Roman', 7.99)]

### 4 - Comptez le nombre total de livres disponibles.
1.   Est-ce une transformation ou une action ?
2.   Le RDD est-il maintenu à ce stade ?



In [7]:
books_rdd.map(lambda x: x[0]).distinct().count()
# Étant donné que la dernière opération count() est une action, le RDD n'est pas maintenu. L'objet renvoyer est un integer.

6

### 5 - Utilisez distinct pour obtenir les genres uniques de livres. Quelle est la nature de l’opération distinct (narrow ou wide) ?

In [8]:
books_rdd.map(lambda x:x[3]).distinct().collect()

['Roman', 'Philosophie']

## Exercice 2: Map-Reduce

Nous avons un jeu de données contenant des informations sur des produits, classés par catégories, ainsi que leur chiffre d'affaires.

- **Colonnes** :
  - `PDT`: Nom du produit
  - `CAT`: Catégorie du produit
  - `CA`: Chiffre d'affaires

In [9]:
sales_rdd = sc.parallelize(
    [("PDT 1", "CAT 1", 200),
     ("PDT 2", "CAT 1", 100),
     ("PDT 3", "CAT 2", 300),
     ("PDT 4", "CAT 2",  350),
    ("PDT 5", "CAT 2", 250)]
)

#

### 1 - Caluler la somme du chiffre d'affaires des produits appartenant à la categorie 2

In [10]:
(
    sales_rdd
    .filter(lambda x : x[1] == "CAT 2")
    .map(lambda x : x[2])
    .reduce(lambda x,y : x+y)
  )

900

### 2 - Calculer la somme du chiffre d'affaires pour chaque catégorie de produits

In [11]:
(
    sales_rdd
    .map(lambda x: (x[1], x[2]))
    .reduceByKey(lambda x,y : x+y)
    .collect()
)

[('CAT 1', 300), ('CAT 2', 900)]


## Exercice 3 : Analyse des Transactions avec Spark SQL

Nous avons un jeu de données transactionnel contenant des informations sur les transactions effectuées par des clients. Chaque transaction est associée à un client et à un produit, avec un montant correspondant.

- **Colonnes** :
  - `transaction_id`: l'identifiant unique de chaque transaction.
  - `customer_id`: l'identifiant du client.
  - `product_id`: l'identifiant du produit.
  - `product_category`: catégorie produit.
  - `amount`: le montant de la transaction.
  - `date`: la date de la transaction.


In [12]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()

### 1 - **Charger les données** dans un DataFrame Spark à partir d'un fichier CSV.

In [14]:
df = spark.read.csv("data td1.csv", header=True, inferSchema=True)
df.limit(5).toPandas()

,transaction_id,customer_id,product_id,product_categorie,boutique,amount,date
0,1,1662,752,Catégorie_50,Boutique_1,50.96,2024-01-14
1,2,2722,386,Catégorie_37,Boutique_21,182.74,2024-02-07
2,3,2298,809,Catégorie_89,Boutique_30,217.60,2023-12-06
3,4,1398,953,Catégorie_33,Boutique_41,401.24,2024-06-17
4,5,4290,90,Catégorie_73,Boutique_6,5.86,2024-06-12


### 2 - Calculer le **chiffre d'affaire généré par chaque categorie de produit** et afficher les top 5 catégories.


In [17]:
(
    df
    .groupby("product_categorie")
    .agg(F.sum("amount").alias("ca"))
    .orderBy(F.desc("ca"))
    .limit(5)
    .toPandas()
)

,product_categorie,ca
0,Catégorie_24,272815.00
1,Catégorie_87,271844.78
2,Catégorie_19,266702.02
3,Catégorie_84,266353.91
4,Catégorie_88,266158.79


### 3 - Filtrer les transactions dont le montant est supérieur à **200** et afficher les résultats.

In [ ]:
(
    df
    .filter(F.col("amount") > 200)
    .limit(5)
    .toPandas()
)

,transaction_id,customer_id,product_id,product_categorie,boutique,amount,date
0,3,2298,809,Catégorie_89,Boutique_30,217.60,2023-12-06
1,4,1398,953,Catégorie_33,Boutique_41,401.24,2024-06-17
2,6,98,305,Catégorie_57,Boutique_25,444.04,2024-05-05
3,9,3391,411,Catégorie_42,Boutique_18,439.39,2024-09-05
4,10,2154,537,Catégorie_27,Boutique_27,345.86,2024-09-08


### 4 - **Trouver le client ayant effectué la plus grande transaction en termes de chiffre d'affaires**

In [18]:
(
    df
    .groupby("customer_id")
    .agg(F.sum("amount").alias("ca"))
    .orderBy(F.desc("ca"))
    .limit(1)
    .toPandas()
)

,customer_id,ca
0,1420,10695.93
